In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

'''
Original code forked from: 

https://www.kaggle.com/dundee2002/rotten-tomatoes-movie-reviews-w-glove-lstm

code and comments added by Leslie Thomas marked LCT 

(Data cleaned, stop words NOT removed; No text blob to break into individual words and did NOT lemmatize) 
'''

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
import os

from nltk.corpus import stopwords # dealing with stop words
from textblob import TextBlob # dealing with spelling correction
from textblob import Word # dealing with lemmatization
from sklearn.feature_extraction.text import TfidfVectorizer # leading with term frequency

from wordcloud import WordCloud
import pandas, numpy, string #xgboost,
import re

import warnings
warnings.filterwarnings("ignore")
print("done")


In [ ]:
train = pd.read_csv('../input/movie-review-sentiment-analysis-kernels-only/train.tsv', sep="\t")
test = pd.read_csv('../input/movie-review-sentiment-analysis-kernels-only/test.tsv', sep="\t")
sub = pd.read_csv('../input/movie-review-sentiment-analysis-kernels-only/sampleSubmission.csv', sep=",")

#EMBEDDING_FILE = ('../input/glovetwitter27b100dtxt/glove.twitter.27B.200d.txt') #, sep="\t")

#print((EMBEDDING_FILE).head())
print("done")


In [ ]:
# Load Data
df = train
pd.set_option('display.max_colwidth', -1)
df.head()

df['Phrase'].tail()

df.info()
print(df.describe())
print(sub.describe())  #LCT
sub_df = test    #   The values for the true test (not validation called test) set for submission 

# inspect sentiment
sns.countplot(df['Sentiment'])

In [ ]:
# text length
df['text_length'] = df['Phrase'].apply(len)
df[['Sentiment','text_length','Phrase']].head()

print(df['text_length'].describe())

df['text_length'].hist(bins=50)

g = sns.FacetGrid(df,col='Sentiment')
g.map(plt.hist,'text_length')

In [ ]:
sns.boxplot(x='Sentiment',y='text_length',data=df,palette='rainbow')

# noted that length of neutral has the shortest phrases and longer towards
# the extremes with the negative side being slightly longer than the positive end.  
# maybe the individuals providing bad reviews really like to rant! 


In [ ]:
#Data cleaning added by LCT and Bruce Pham


text1 = df['Phrase'].to_string()  # need to convert to string to create a word cloud


###########################  LCT clean data #######################################
def clean_text(text):
     #text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
     text = re.sub('[%s]' % re.escape(string.digits), '', text)
     text = re.sub('[%s]' % re.escape(' +'), ' ', text)
     text = text.replace(' n\'t', ' not')   # LCT convert contractions
     text = text.replace('it \'s', 'it is')
     text = text.replace('there \'s', 'there is')
     text = text.replace('he \'s', 'he is')
     text = text.replace('she \'s', 'she is')
     text = text.replace('what \'s', 'what is')
     text = text.replace('that \'s', 'that is')
     text = text.replace(' \'s', '')
     text = text.replace('\'s', '')
     text = text.replace('s \'', '')
     text = text.replace('-lrb-', '')  # LCT remove left and right brackets as they do not add sentiment
     text = text.replace('-LRB-', '')
     text = text.replace('-rrb-', '')
     text = text.replace('-RRB-', '')
     text = text.lower()
     text = text.strip()
     return text

text1 = clean_text(text1)

text = text1
#############################################################################
# word cloud

wordcloud = WordCloud(
        relative_scaling=0.5,
        stopwords=set(stopwords.words('english'))).generate(text)

plt.figure(figsize=(12,12))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

# notice the words Film and Movie are prominent in the word cloud

In [ ]:
################ CLEAN TRAIN AND TEST #####################################################

train['Phrase_clean_text'] = train['Phrase'].apply(lambda x: clean_text(x))
sub_df['Phrase_clean_text'] = sub_df['Phrase'].apply(lambda x: clean_text(x))
###################################################

# stopwords and lemmatizing based on : 
#https://www.kaggle.com/up201204722/movie-sentiment-analysis
############## TAKE OUT STOP WORDS
#Decided not to remove stop words as some phrases are only one word and the reviews are filled with stop words
#Also, it deteriorated the performance of the model 

#del stopwords 

#stop = stopwords.words('english')

#newStopWords = ['movie','film']  # LCT add stop words that don't differentiate reviews
#stop.extend(newStopWords)
#len(stop)

#train['Phrase_clean_text'] = train['Phrase_clean_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#sub_df['Phrase_clean_text'] = sub_df['Phrase_clean_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))


# Also decided not to use text blob to break phrases into individual words as some phrases are only one word long 
# tried this but it made performance worse 


In [ ]:
######  OMITTING LEMMATIZING THIS TIME TO SEE IMPACT 


#from #https://www.kaggle.com/up201204722/movie-sentiment-analysis
#Lemmatization Lemmatization is a more effective option than stemming because
# it converts the word into its root word, rather than just stripping the suffixes. 
#It makes use of the vocabulary and does a morphological analysis to obtain the root word.
# Therefore, we usually prefer using lemmatization over stemming.
 ###############

#train['Phrase_clean_text'] = train['Phrase_clean_text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
#sub_df['Phrase_clean_text'] = sub_df['Phrase_clean_text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

print("complete")

In [ ]:
# Encode Categorical Variable
from keras.utils import to_categorical
X = df['Phrase_clean_text']   # modified
y = to_categorical(df['Sentiment'])
num_classes = df['Sentiment'].nunique()
y

# LCT  prepare the data provided for test (not to confused with the validation set
# called test) 

X_sub = sub_df['Phrase_clean_text']  # added by LCT # modified
########################################################

In [ ]:
seed = 101 # fix random seed for reproducibility
np.random.seed(seed)

# Spilt Train Test sets


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=seed)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


In [ ]:
########################################################

# Tokenize Text
from keras.preprocessing.text import Tokenizer
max_features = 15000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# tokenize X_sub = test set for submission #LCT
X_sub = tokenizer.texts_to_sequences(X_sub)  #LCT

totalNumWords = [len(one_comment) for one_comment in X_train]
plt.hist(totalNumWords,bins = 30)
plt.show()

from keras.preprocessing import sequence
max_words = 30 #max(totalNumWords)
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
print(X_train.shape,X_test.shape)

# pad test set for submission   #LCT 
X_sub = sequence.pad_sequences(X_sub, maxlen=max_words)


import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

batch_size = 256
epochs = 5

def get_model(max_features, embed_dim):
    np.random.seed(seed)
    K.clear_session()
    model = Sequential()
    model.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1]))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

def model_train(model):
    # train the model
    model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                          epochs=epochs, batch_size=batch_size, verbose=2)
    # plot train history
    plot_model_history(model_history)
    
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show() 
    
    
def model_evaluate(): 
    # predict class with test set
    y_pred_test =  model.predict_classes(X_test, batch_size=batch_size, verbose=0)
    print('Accuracy:\t{:0.1f}%'.format(accuracy_score(np.argmax(y_test,axis=1),y_pred_test)*100))
    
    #classification report
    print('\n')
    print(classification_report(np.argmax(y_test,axis=1), y_pred_test))

    #confusion matrix
    confmat = confusion_matrix(np.argmax(y_test,axis=1), y_pred_test)

    fig, ax = plt.subplots(figsize=(4, 4))
    ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.tight_layout()    

# train the model
max_features = 15000
embed_dim = 100
model = get_model(max_features, embed_dim)
model_train(model)

In [ ]:
# Look at that elbow at the end of the second epoch in graph above for the validation set (orange line)
# It appears that additioal epochs beyond 2 do not add value in the model. The improvement in model accuracy even for the training set (blue line)
# even starts to drop off here. 

# evaluate model with test set
model_evaluate()


#############################################################

# LCT predicts to submit for this model

y_pred=model.predict_classes(X_sub)  # predict for prepared test set to submit
y_pred

len(y_pred)

testPred= np.array(y_pred).transpose().tolist()

len(testPred)

sub.Sentiment=testPred
sub.head()

sub.to_csv('keras_dundee_clean_only_No_Lemmat.csv',index=False)

In [ ]:
############KERAS DEEP LEARNING MODEL with Twitter Embedding ###############

def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
    
def get_embed_mat(EMBEDDING_FILE, max_features=20000):
    # word vectors
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding='utf8'))
    print('Found %s word vectors.' % len(embeddings_index))

    # embedding matrix
    word_index = tokenizer.word_index
    num_words = min(max_features, len(word_index) + 1)
    all_embs = np.stack(embeddings_index.values()) #for random init
    embedding_matrix = np.random.normal(all_embs.mean(), all_embs.std(), 
                                        (num_words, embed_dim))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    max_features = embedding_matrix.shape[0]
    
    return max_features, embedding_matrix

def get_model(max_features, embed_dim, embedding_matrix):
    np.random.seed(seed)
    K.clear_session()
    model = Sequential()
    model.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1],
                       weights=[embedding_matrix]))#,trainable=False
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

EMBEDDING_FILE = '../input/glovetwitter27b100dtxt/glove.twitter.27B.200d.txt'






In [ ]:

embed_dim = 200 #word vector dim
max_features, embedding_matrix = get_embed_mat(EMBEDDING_FILE)

# train the model
model = get_model(max_features, embed_dim, embedding_matrix)
model_train(model)

In [ ]:
# evaluate model with test set
model_evaluate()

# LCT predicts to submit

y_pred=model.predict_classes(X_sub)  # predict for prepared test set to submit
y_pred

len(y_pred)

testPred= np.array(y_pred).transpose().tolist()

len(testPred)

sub.Sentiment=testPred
sub.head()

sub.to_csv('subm_twit_clean_No_Lemma.csv',index=False)
